In [2]:
import pandas as pd
from rasterstats import zonal_stats
import geopandas as gpd
import os

In [3]:
# Load districts
simplified_uganda_districts = gpd.read_file(
    "C:/Users/Administrator/OneDrive/Desktop/rainfall_prediction/uganda_administrative_boundaries/simplified_uganda_districts.shp"
)

simplified_uganda_districts

,ADM2_EN,ADM1_EN,geometry
0,Abim,Northern,"POLYGON ((33.58099 3.14912, 33.58555 3.14702, ..."
1,Adjumani,Northern,"POLYGON ((32.04597 3.58896, 32.04578 3.58875, ..."
2,Agago,Northern,"POLYGON ((33.41652 3.30434, 33.41701 3.30403, ..."
3,Alebtong,Northern,"POLYGON ((33.03629 2.5013, 33.03664 2.50075, 3..."
4,Amolatar,Northern,"POLYGON ((32.9014 1.8057, 32.90231 1.80568, 32..."
...,...,...,...
130,Ssembabule,Central,"POLYGON ((31.16746 0.23304, 31.16911 0.23182, ..."
131,Tororo,Eastern,"POLYGON ((34.09371 0.90204, 34.09421 0.90204, ..."
132,Wakiso,Central,"POLYGON ((32.58831 0.57438, 32.58969 0.57431, ..."
133,Yumbe,Northern,"POLYGON ((31.21323 3.79479, 31.21367 3.79444, ..."


In [4]:
# Process raster files
rainfall_data = []

# Loop through raster files in the specified directory
for file in sorted(
    f
    for f in os.listdir(
        "C:/Users/Administrator/OneDrive/Desktop/rainfall_prediction/rainfall/"
    )
    if f.endswith(".tif")
):

    # Extract year and month from filename
    try:
        parts = file.split(".")
        year = parts[-3]
        month = parts[-2]
        date = f"{year}-{month}-01"
        print(f"Processing file: {file} for date: {date}")

    except IndexError:
        print(f"Filename {file} does not match expected format. Skipping.")
        continue

    # Compute zonal statistics
    stats = zonal_stats(
        simplified_uganda_districts,
        f"C:/Users/Administrator/OneDrive/Desktop/rainfall_prediction/rainfall/{file}",
        stats=["mean"],
        geojson_out=True,
    )

    # Extract mean rainfall and district names
    for feature in stats:
        district = feature["properties"]["ADM2_EN"]
        rainfall = feature["properties"]["mean"]
        region = feature["properties"]["ADM1_EN"]

        rainfall_data.append(
            {
                "district": district,
                "region": region,
                "date": date,
                "mean_rainfall": rainfall,
            }
        )
    print(f"Completed processing for date: {date}")

# Create DataFrame from collected data
rainfall_df = pd.DataFrame(rainfall_data)
print(f"Created rainfall DataFrame with {len(rainfall_df)} records.")

rainfall_df

Processing file: chirps-v2.0.1981.01.tif for date: 1981-01-01


c:\Program Files\Python311\Lib\site-packages\rasterstats\io.py:335: NodataWarning: Setting nodata to -999; specify nodata explicitly
  warnings.warn(


Completed processing for date: 1981-01-01
Processing file: chirps-v2.0.1981.02.tif for date: 1981-02-01
Completed processing for date: 1981-02-01
Processing file: chirps-v2.0.1981.03.tif for date: 1981-03-01
Completed processing for date: 1981-03-01
Processing file: chirps-v2.0.1981.04.tif for date: 1981-04-01
Completed processing for date: 1981-04-01
Processing file: chirps-v2.0.1981.05.tif for date: 1981-05-01
Completed processing for date: 1981-05-01
Processing file: chirps-v2.0.1981.06.tif for date: 1981-06-01
Completed processing for date: 1981-06-01
Processing file: chirps-v2.0.1981.07.tif for date: 1981-07-01
Completed processing for date: 1981-07-01
Processing file: chirps-v2.0.1981.08.tif for date: 1981-08-01
Completed processing for date: 1981-08-01
Processing file: chirps-v2.0.1981.09.tif for date: 1981-09-01
Completed processing for date: 1981-09-01
Processing file: chirps-v2.0.1981.10.tif for date: 1981-10-01
Completed processing for date: 1981-10-01
Processing file: chirp

,district,region,date,mean_rainfall
0,Abim,Northern,1981-01-01,7.921172
1,Adjumani,Northern,1981-01-01,6.550826
2,Agago,Northern,1981-01-01,5.667293
3,Alebtong,Northern,1981-01-01,15.642982
4,Amolatar,Northern,1981-01-01,16.605842
...,...,...,...,...
72625,Ssembabule,Central,2025-10-01,115.724401
72626,Tororo,Eastern,2025-10-01,191.841295
72627,Wakiso,Central,2025-10-01,104.733953
72628,Yumbe,Northern,2025-10-01,172.643377


In [5]:
rainfall_df.to_csv("uganda_districts_monthly_rainfall.csv", index=False)